In [9]:
# train.py
import os
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ---- SETTINGS ----
DATA_DIR = r"F:\Work\ORANGE MODELS\brain_tumor_dataset\train"              # <-- put your dataset folder path here
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123
EPOCHS = 12
MODEL_DIR = "brain_ct_classifier" # will save SavedModel folder
H5_PATH = "brain_ct_classifier.h5"
LABEL_MAP_PATH = "label_map.json"
# -------------------

# reproducible-ish
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Dataset folder not found at '{DATA_DIR}'. Expected structure:\n{DATA_DIR}/healthy/*\n{DATA_DIR}/tumor/*")

AUTOTUNE = tf.data.AUTOTUNE

# 1) create train/val datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

class_names = train_ds.class_names
print("Class names (index -> name):", list(enumerate(class_names)))

# save label map
with open(LABEL_MAP_PATH, "w") as f:
    json.dump(class_names, f)
print(f"Saved label map to '{LABEL_MAP_PATH}'")

# 2) dataset performance
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 3) build model
# We'll perform scaling inside the model: scale pixels from [0,255] -> [-1,1]
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
])

base_model = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                         include_top=False,
                         weights="imagenet")
base_model.trainable = False  # freeze for initial training

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = layers.Rescaling(1./127.5, offset=-1)(inputs)   # IMPORTANT: maps [0,255] -> [-1,1]
x = data_augmentation(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# 4) callbacks
checkpoint_cb = ModelCheckpoint(filepath=os.path.join(MODEL_DIR, "best_model.h5"),
                                save_best_only=True, monitor="val_accuracy", mode="max")
earlystop_cb = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7)

os.makedirs(MODEL_DIR, exist_ok=True)

# 5) train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr]
)

# 6) evaluate on validation set (sanity)
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc:.4f}, loss: {val_loss:.4f}")

# 7) save final model (both SavedModel folder and .h5 for convenience)
print("Saving model...")
model.save("brain_ct_classifier.h5")                 # SavedModel format
model.save(H5_PATH)                   # .h5 copy
print(f"Saved model to '{MODEL_DIR}/' and '{H5_PATH}'.")


Found 8036 files belonging to 2 classes.
Using 6429 files for training.
Found 8036 files belonging to 2 classes.
Using 1607 files for validation.
Class names (index -> name): [(0, 'Healthy'), (1, 'Tumor')]
Saved label map to 'label_map.json'


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 164,226 (641.51 KB)

...

201/201 ━━━━━━━━━━━━━━━━━━━━ 59s 295ms/step - accuracy: 0.9440 - loss: 0.1421 - val_accuracy: 0.9564 - val_loss: 0.1185 - learning_rate: 1.0000e-04
Epoch 5/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 61s 305ms/step - accuracy: 0.9468 - loss: 0.1412 - val_accuracy: 0.9496 - val_loss: 0.1295 - learning_rate: 1.0000e-04
Epoch 6/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.9466 - loss: 0.1365

201/201 ━━━━━━━━━━━━━━━━━━━━ 59s 295ms/step - accuracy: 0.9466 - loss: 0.1365 - val_accuracy: 0.9596 - val_loss: 0.1117 - learning_rate: 1.0000e-04
Epoch 7/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - accuracy: 0.9546 - loss: 0.1220

201/201 ━━━━━━━━━━━━━━━━━━━━ 59s 292ms/step - accuracy: 0.9546 - loss: 0.1220 - val_accuracy: 0.9614 - val_loss: 0.1062 - learning_rate: 1.0000e-04
Epoch 8/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.9576 - loss: 0.1122

201/201 ━━━━━━━━━━━━━━━━━━━━ 61s 303ms/step - accuracy: 0.9576 - loss: 0.1122 - val_accuracy: 0.9633 - val_loss: 0.1059 - learning_rate: 1.0000e-04
Epoch 9/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 289ms/step - accuracy: 0.9550 - loss: 0.1106 - val_accuracy: 0.9583 - val_loss: 0.1134 - learning_rate: 1.0000e-04
Epoch 10/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 291ms/step - accuracy: 0.9561 - loss: 0.1141 - val_accuracy: 0.9589 - val_loss: 0.1099 - learning_rate: 1.0000e-04
Epoch 11/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 290ms/step - accuracy: 0.9530 - loss: 0.1102 - val_accuracy: 0.9571 - val_loss: 0.1123 - learning_rate: 1.0000e-04
Epoch 12/12
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 288ms/step - accuracy: 0.9585 - loss: 0.1052 - val_accuracy: 0.9633 - val_loss: 0.0982 - learning_rate: 5.0000e-05
51/51 ━━━━━━━━━━━━━━━━━━━━ 11s 208ms/step - accuracy: 0.9644 - loss: 0.1022


Validation accuracy: 0.9633, loss: 0.0982
Saving model...
Saved model to 'brain_ct_classifier/' and 'brain_ct_classifier.h5'.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# ---------------- SETTINGS ----------------
MODEL_PATH = r'brain_ct_classifier.h5'   # or "brain_ct_classifier.h5"
LABEL_MAP = ["healthy", "tumor"]           # order must match training
IMG_SIZE = (224, 224)                      # must match training
TEST_IMAGE = r"F:\Work\ORANGE MODELS\cancer detection\Healthy\1 no.jpeg"
# ------------------------------------------

# Load model
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded")

# Load and preprocess image
img = image.load_img(TEST_IMAGE, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)   # add batch dimension

# ⚠️ IMPORTANT: only normalize if you did NOT use Rescaling in training
# If you used layers.Rescaling(1./255) in your model, remove the next line
# img_array = img_array / 255.0

# Predict
preds = model.predict(img_array)
probs = preds[0]
pred_idx = np.argmax(probs)
confidence = probs[pred_idx]

print("\n--- Prediction ---")
for label, p in zip(LABEL_MAP, probs):
    print(f"{label}: {p*100:.2f}%")

print(f"\n➡️ Final Prediction: {LABEL_MAP[pred_idx]} (confidence: {confidence*100:.2f}%)")


✅ Model loaded
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step

--- Prediction ---
healthy: 93.30%
tumor: 6.70%

➡️ Final Prediction: healthy (confidence: 93.30%)
